In [304]:
from avcv.all import *
from avcv.coco import *
input_dirs = [
    '/data/RLDD/',
    '/data/Data_HonTre/symlink_videos/'
]

In [305]:
csv_paths = []
for input_dir in input_dirs:
    csv_paths += glob(osp.join(input_dir, '*/cropped_face.csv'))
    print(input_dir, len(csv_paths))

/data/RLDD/ 154
/data/Data_HonTre/symlink_videos/ 5479


In [306]:
sample_rate = 3
def f(csv_path):
    sample_rate
    img_dir = osp.join(osp.dirname(csv_path), 'cropped_face')
    df = pd.read_csv(csv_path)
    df = df[['path']]
    df['path'] = df.path.apply(lambda x: osp.join(img_dir, x))
    df['dataset'] = df.path.apply(lambda x: x.split('/')[2])
    df['video'] = df.path.apply(lambda x: x.split('/')[4])
    return df[0::sample_rate]

In [307]:
from multiprocessing import Pool
from tqdm import tqdm
with Pool(10) as p:
    it = p.imap(f, csv_paths)
    total = len(csv_paths)
    dfs = tqdm(list(it))
df = pd.concat(dfs)
df['index'] = range(len(df))
df = df.set_index('index')    

In [310]:
df.groupby('dataset').size()

dataset
Data_HonTre    1136700
RLDD            892754
dtype: int64

In [311]:
import shutil

In [312]:
row = df.iloc[0]

In [315]:
# df

In [341]:
out_dir = './data/cropfaces_v2/'
def f(inp):
    index, row = inp
    sub_dir = index%500
    new_path = osp.join(out_dir, f'{sub_dir:04d}/{index:06d}.jpg')
    # mmcv.mkdir_or_exist(osp.dirname(new_path))
    try:
        os.makedirs(osp.dirname(new_path))
    except:
        pass
    
    shutil.copy(row.path, new_path)

In [342]:
with Pool(60) as p:
    it = p.imap(f, df.iterrows())
    total = len(df)
    r = list(tqdm(it, total=total))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2029454/2029454 [09:11<00:00, 3682.42it/s]
